# 🔧 ETAPA UNIFICADA: Carregamento do Manifesto, do Pipeline e Extração do Signature

Neste bloco, vamos:
1️⃣ Verificar a existência e ler o manifesto v1-final.  
2️⃣ Extrair o `run_id` e carregar o pipeline congelado via MLflow.  
3️⃣ Recuperar e imprimir o schema de entrada (`signature.inputs`) do modelo.

Objetivo: garantir que `modelo_congelado` esteja sempre definido antes de extrair seu `signature`, tudo num só comando rastreável.


In [4]:
import os
import json
import mlflow.pyfunc
from mlflow.types import DataType

# 1️⃣ Verificação formal do manifesto
manifesto_path = "/workspace/models/congelados/manifesto_modelo_rf_v1.json"
if not os.path.exists(manifesto_path):
    raise FileNotFoundError(f"🚫 Manifesto não encontrado: {manifesto_path}")

# 2️⃣ Leitura literal do JSON
with open(manifesto_path, "r") as f:
    manifesto = json.load(f)

# 3️⃣ Extração e validação do run_id
run_id = manifesto.get("run_id")
if not isinstance(run_id, str):
    raise KeyError("🚫 'run_id' ausente ou inválido no manifesto")

# 4️⃣ Definição e carregamento do modelo congelado
mlflow.set_tracking_uri("file:/workspace/.mlruns")
model_uri = f"runs:/{run_id}/model"
modelo_congelado = mlflow.pyfunc.load_model(model_uri)

print("✅ Manifesto e modelo carregados. run_id:", run_id)
print("🔗 model_uri:", model_uri)

# 5️⃣ COMANDO PRINCIPAL: extração e impressão do signature.inputs
print("\n🗂️ signature.inputs:")
for field in modelo_congelado.metadata.signature.inputs:
    print(f"• {field.name:<50} | Tipo MLflow: {field.type}")


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Manifesto e modelo carregados. run_id: 7077ebfbf696487384bd5a59034170c5
🔗 model_uri: runs:/7077ebfbf696487384bd5a59034170c5/model

🗂️ signature.inputs:
• Age                                                | Tipo MLflow: DataType.double
• Age_Binned                                         | Tipo MLflow: DataType.string
• Amount_invested_monthly                            | Tipo MLflow: DataType.double
• Amount_invested_monthly_Binned                     | Tipo MLflow: DataType.string
• Amount_invested_monthly_Binned_High                | Tipo MLflow: DataType.boolean
• Amount_invested_monthly_Binned_Low                 | Tipo MLflow: DataType.boolean
• Amount_invested_monthly_Binned_Moderate            | Tipo MLflow: DataType.boolean
• Annual_Income                                      | Tipo MLflow: DataType.double
• Annual_Income_Binned                               | Tipo MLflow: DataType.string
• Changed_Credit_Limit                               | Tipo MLflow: DataType.double
• C

# 🔧 ETAPA 3 (CORRIGIDA): Detecção de Pydantic Models na API

Neste bloco, vamos:
1️⃣ Percorrer os arquivos `.py` do diretório da API.
2️⃣ Detectar classes que herdam de `pydantic.BaseModel`.
3️⃣ Imprimir cada classe encontrada e seus campos (nome e tipo).

Objetivo: descobrir programaticamente o nome correto do modelo de requisição e seu schema, sem suposições.


In [6]:
import os
import ast
import importlib.util
from pydantic import BaseModel

# Diretório raiz da API (ajuste se necessário)
api_dir = "/workspace/api"  # ou o caminho correto do seu código

models = []

# 1️⃣ Percorre arquivos .py na pasta da API
for root, _, files in os.walk(api_dir):
    for fname in files:
        if not fname.endswith(".py"):
            continue
        path = os.path.join(root, fname)
        # 2️⃣ Parse AST para encontrar classes BaseModel
        with open(path, "r") as f:
            tree = ast.parse(f.read(), filename=path)
        for node in tree.body:
            if isinstance(node, ast.ClassDef):
                for base in node.bases:
                    if getattr(base, 'attr', None) == 'BaseModel' or getattr(base, 'id', None) == 'BaseModel':
                        models.append((path, node.name))

# 3️⃣ Impressão das classes Pydantic encontradas
if not models:
    print("⚠️ Nenhuma classe BaseModel encontrada em", api_dir)
else:
    for file_path, model_name in models:
        print(f"• Arquivo: {file_path}")
        print(f"  Classe: {model_name}")
        # Importa dinamicamente e imprime campos
        spec = importlib.util.spec_from_file_location("mod", file_path)
        mod = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(mod)
        cls = getattr(mod, model_name)
        if issubclass(cls, BaseModel):
            print("  Campos:")
            for name, field in cls.__fields__.items():
                print(f"    - {name}: {field.outer_type_}")
        print()


⚠️ Nenhuma classe BaseModel encontrada em /workspace/api


In [7]:
# 🔧 ETAPA 3 (LOCALIZAÇÃO): Scan de todo o projeto em busca de classes Pydantic
"""
Esta célula percorre recursivamente o diretório /workspace,
analisa cada .py e imprime o caminho e nome de toda classe que herda de BaseModel.
"""
import os, ast

# comando principal: varre o repositório
for root, _, files in os.walk("/workspace"):
    for fname in files:
        if not fname.endswith(".py"):
            continue
        path = os.path.join(root, fname)
        with open(path, "r", encoding="utf-8") as f:
            tree = ast.parse(f.read(), filename=path)
        for node in tree.body:
            if isinstance(node, ast.ClassDef):
                for base in node.bases:
                    if getattr(base, "id", "") == "BaseModel" or getattr(base, "attr", "") == "BaseModel":
                        print(f"• {path}  ->  class {node.name}")


• /workspace/api.py  ->  class Item


# 🔧 ETAPA 3 (CORRIGIDO): Extração do Schema da API via Pydantic `Item`

Neste bloco, vamos:
1️⃣ Importar `Item` correto da API.  
2️⃣ Iterar sobre `Item.model_fields` para obter cada campo e sua annotation.  
3️⃣ Imprimir nome do campo e tipo, garantindo compatibilidade com Pydantic v2.

Objetivo: capturar fielmente o schema que a API espera, sem erros de atributo.


In [9]:
# 🔧 ETAPA 3: Extração de campos de Item (Pydantic v2) na API

from api import Item  # modelo correto encontrado

# COMANDO PRINCIPAL: impressão dos campos e tipos via model_fields
for nome, field in Item.model_fields.items():
    print(f"• {nome:<30} | Tipo API (Pydantic): {field.annotation}")

# Em fallback, poderíamos usar get_type_hints(Item) se model_fields não existir:
# 
# from typing import get_type_hints
# for nome, typ in get_type_hints(Item).items():
#     print(f"• {nome:<30} | Tipo API (anotação): {typ}")


• data                           | Tipo API (Pydantic): typing.List[typing.Dict[str, typing.Any]]


# 🔧 ETAPA 4 (CORRIGIDA): Detecção de Widgets no Formulário Streamlit

Neste bloco, vamos:
1️⃣ Parsear `/workspace/streamlit_app_V1.py` recursivamente.  
2️⃣ Usar `ast.walk` para localizar **todos** os blocos `with st.form(...)`.  
3️⃣ Dentro de cada bloco de formulário, identificar chamadas a widgets `st.*_input`, `st.selectbox`, etc.  
4️⃣ Extrair o parâmetro `label` (primeiro argumento literal) e o valor de `key` (se informado).  
5️⃣ Imprimir nome do widget, label, key e arquivo para mapeamento completo.

Objetivo: capturar fielmente quais campos o Streamlit coleta no formulário.


In [12]:
import ast

# 1️⃣ Carrega e parseia o código
path = "/workspace/streamlit_app_V1.py"
with open(path, "r", encoding="utf-8") as f:
    tree = ast.parse(f.read(), filename=path)

print(f"Campos dentro de st.form em {path}:")

# 2️⃣ Varre toda a AST em busca de blocos with st.form(...)
for node in ast.walk(tree):
    if isinstance(node, ast.With):
        for item in node.items:
            expr = item.context_expr
            if isinstance(expr, ast.Call) and getattr(expr.func, "attr", "") == "form":
                # 3️⃣ Dentro do form, varre chamadas a widgets
                for stmt in node.body:
                    # procura chamadas st.widget(...)
                    for call in ast.walk(stmt):
                        if isinstance(call, ast.Call) and isinstance(call.func, ast.Attribute):
                            if getattr(call.func.value, "id", "") == "st":
                                widget = call.func.attr
                                # extrai label (first arg literal)
                                label = None
                                if call.args and isinstance(call.args[0], ast.Constant):
                                    label = call.args[0].value
                                # extrai key (kwarg 'key')
                                key = None
                                for kw in call.keywords:
                                    if kw.arg == "key" and isinstance(kw.value, ast.Constant):
                                        key = kw.value.value
                                print(f"• st.{widget:<15} | label: {label!r} | key: {key!r}")


Campos dentro de st.form em /workspace/streamlit_app_V1.py:
• st.selectbox       | label: 'Idade' | key: None
• st.selectbox       | label: 'Ocupação' | key: None
• st.selectbox       | label: 'Perfil de Crédito' | key: None
• st.selectbox       | label: 'Pagamento Mínimo' | key: None
• st.number_input    | label: 'Idade do Histórico de Crédito' | key: None
• st.selectbox       | label: 'Salário Mensal' | key: None
• st.selectbox       | label: 'Nº de Cartões' | key: None
• st.selectbox       | label: 'Pagamentos Atrasados' | key: None
• st.selectbox       | label: 'Renda Anual' | key: None
• st.selectbox       | label: 'Dívida' | key: None
• st.selectbox       | label: 'Taxa de Juros' | key: None
• st.selectbox       | label: 'Mudança de Limite' | key: None
• st.selectbox       | label: 'Histórico Binned' | key: None
• st.selectbox       | label: 'Utilização de Crédito' | key: None
• st.selectbox       | label: 'Dias de Atraso' | key: None
• st.selectbox       | label: 'Balanço Mensal

# 🔧 ETAPA 5 (CORRIGIDA): Criação do diretório e salvamento da tabela de mapeamento

Nesta célula, vamos:
1️⃣ Garantir que o diretório `/workspace/docs` exista.
2️⃣ Salvar o DataFrame de mapeamento em `/workspace/docs/variable_mapping.csv`.

Objetivo: persistir a “single source of truth” sem erros de diretório.


In [14]:
import os
import pandas as pd
from mlflow.types import DataType

# (Reutilize o df_map já construído na célula anterior)

# 1️⃣ Criação do diretório se necessário
output_dir = "/workspace/docs"
os.makedirs(output_dir, exist_ok=True)

# 2️⃣ COMANDO PRINCIPAL: salvar o CSV
output_path = os.path.join(output_dir, "variable_mapping.csv")
df_map.to_csv(output_path, index=False)

# Impressão de verificação
print("✅ Tabela de mapeamento inicial salva em:", output_path)
print(df_map.head(10))


✅ Tabela de mapeamento inicial salva em: /workspace/docs/variable_mapping.csv
                              pipeline_var pipeline_type api_field  \
0                                      Age        double      data   
1                               Age_Binned        string      data   
2                  Amount_invested_monthly        double      data   
3           Amount_invested_monthly_Binned        string      data   
4      Amount_invested_monthly_Binned_High       boolean      data   
5       Amount_invested_monthly_Binned_Low       boolean      data   
6  Amount_invested_monthly_Binned_Moderate       boolean      data   
7                            Annual_Income        double      data   
8                     Annual_Income_Binned        string      data   
9                     Changed_Credit_Limit        double      data   

               api_type streamlit_label streamlit_widget  
0  List[Dict[str, Any]]            None             None  
1  List[Dict[str, Any]]          

In [15]:
# 🔧 ETAPA 3: Criação do Módulo de Mapeamento de Variáveis

# Descrição:
# Esta célula gera automaticamente o arquivo de configuração `mapping.py`,
# contendo o dicionário VARIABLE_MAP com todas as variáveis do pipeline.
# Cada entrada vem inicializada com label, widget e api = None,
# para que você preencha em seguida.

import os

# 1️⃣ Coleta das variáveis do pipeline
pipeline_vars = [field.name for field in modelo_congelado.metadata.signature.inputs]

# 2️⃣ Prepara o dicionário esqueleto
map_entries = {
    var: {"label": None, "widget": None, "api": None}
    for var in pipeline_vars
}

# 3️⃣ Gera o arquivo mapping.py
mapping_path = "/workspace/mapping.py"
os.makedirs(os.path.dirname(mapping_path), exist_ok=True)
with open(mapping_path, "w", encoding="utf-8") as f:
    f.write("# Auto-generated mapping module\n")
    f.write("VARIABLE_MAP = {\n")
    for var, cfg in map_entries.items():
        f.write(f"    '{var}': {cfg},\n")
    f.write("}\n")

# COMANDO PRINCIPAL: verificação de criação
if os.path.exists(mapping_path):
    print(f"✅ mapping.py criado com sucesso em: {mapping_path}")
else:
    raise FileNotFoundError(f"❌ Falha ao criar mapping.py em: {mapping_path}")

print("🔗 Agora abra mapping.py e preencha os campos 'label', 'widget' e 'api' para cada variável.")


✅ mapping.py criado com sucesso em: /workspace/mapping.py
🔗 Agora abra mapping.py e preencha os campos 'label', 'widget' e 'api' para cada variável.


🔧 ETAPA 4: Geração Dinâmica do Modelo Pydantic a partir de VARIABLE_MAP
Nesta célula, vamos criar automaticamente a classe RequestPayload do Pydantic com base no dicionário VARIABLE_MAP.
Cada campo do modelo terá o mesmo nome definido em "api" e seu tipo será inferido a partir do tipo MLflow (double/long → float, string → str, boolean → bool).

In [16]:
from pydantic import create_model
from typing import Any, Dict, List
from mlflow.types import DataType
from mapping import VARIABLE_MAP

# COMANDO PRINCIPAL: criação dinâmica do modelo RequestPayload
fields = {}
for var, cfg in VARIABLE_MAP.items():
    api_name = cfg["api"]
    if api_name:
        # encontra o tipo MLflow para essa variável
        field_type = next(
            f.type for f in modelo_congelado.metadata.signature.inputs if f.name == var
        )
        # mapeia DataType → Python type
        if field_type in (DataType.double, DataType.long):
            py_type = float
        elif field_type == DataType.boolean:
            py_type = bool
        else:
            py_type = str
        fields[api_name] = (py_type, ...)
        
RequestPayload = create_model("RequestPayload", **fields)

# Verificação
print("✅ RequestPayload criado com campos:", list(RequestPayload.__fields__.keys()))


✅ RequestPayload criado com campos: []


/tmp/ipykernel_11246/1286408288.py:27: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print("✅ RequestPayload criado com campos:", list(RequestPayload.__fields__.keys()))


🔧 ETAPA 4.1: Geração do Modelo Pydantic com Fallback de api
Nesta célula, ajustamos a geração do RequestPayload para que, se em VARIABLE_MAP o campo "api" estiver None, seja usado o próprio nome da variável do pipeline como fallback. Isso garante que todos os inputs do signature sejam incluídos no modelo.

In [17]:
from pydantic import create_model
from mlflow.types import DataType
from mapping import VARIABLE_MAP

# COMANDO PRINCIPAL: criação dinâmica do modelo RequestPayload com fallback
fields = {}
for var, cfg in VARIABLE_MAP.items():
    api_name = cfg["api"] or var
    # busca o tipo MLflow no signature
    field_type = next(
        f.type for f in modelo_congelado.metadata.signature.inputs if f.name == var
    )
    # mapeia DataType para tipo Python
    if field_type in (DataType.double, DataType.long):
        py_type = float
    elif field_type == DataType.boolean:
        py_type = bool
    else:
        py_type = str
    fields[api_name] = (py_type, ...)
    
RequestPayload = create_model("RequestPayload", **fields)

# Verificação
print("✅ RequestPayload agora criado com campos:", list(RequestPayload.model_fields.keys()))


✅ RequestPayload agora criado com campos: ['Age', 'Age_Binned', 'Amount_invested_monthly', 'Amount_invested_monthly_Binned', 'Amount_invested_monthly_Binned_High', 'Amount_invested_monthly_Binned_Low', 'Amount_invested_monthly_Binned_Moderate', 'Annual_Income', 'Annual_Income_Binned', 'Changed_Credit_Limit', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_History_Age_Months', 'Credit_Mix', 'Credit_Utilization_Ratio', 'Credit_Utilization_Ratio_Binned', 'Credit_Utilization_Ratio_Binned_High', 'Credit_Utilization_Ratio_Binned_Low', 'Credit_Utilization_Ratio_Binned_Moderate', 'Delay_from_due_date', 'Delay_from_due_date_Binned', 'Interest_Rate', 'Interest_Rate_Binned', 'Month_August', 'Month_February', 'Month_January', 'Month_July', 'Month_June', 'Month_March', 'Month_May', 'Month_November', 'Month_October', 'Month_September', 'Monthly_Balance', 'Monthly_Balance_Binned', 'Monthly_Balance_Binned_High', 'Monthly_Balance_Binned_Low', 'Monthly_Balance_B

🔧 ETAPA 5: Geração Dinâmica do Formulário Streamlit a partir de VARIABLE_MAP
Nesta célula, vamos construir o formulário st.form automaticamente com base em VARIABLE_MAP.
Cada variável será renderizada usando o widget definido em "widget" e com o rótulo "label".
Objetivo: manter UI e API totalmente sincronizadas com a mesma fonte de mapeamento.

python
Copiar
Editar


In [18]:
import streamlit as st
from mapping import VARIABLE_MAP

# COMANDO PRINCIPAL: construção do formulário dinâmico
with st.form("credito_form"):
    inputs = {}
    for var, cfg in VARIABLE_MAP.items():
        label = cfg["label"]
        widget = cfg["widget"]
        key = var
        if label is None or widget is None:
            continue  # pula variáveis não mapeadas
        if widget == "selectbox":
            # você deve definir get_options(var) conforme o domínio de cada campo
            inputs[var] = st.selectbox(label, options=get_options(var), key=key)
        elif widget == "number_input":
            inputs[var] = st.number_input(label, key=key)
        # adicione outros tipos de widget conforme cfg["widget"]
    if st.form_submit_button("Enviar"):
        st.json(inputs)


2025-07-23 20:00:21.140 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:00:21.141 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:00:21.141 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:00:21.141 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:00:21.141 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:00:21.141 WARNING streamlit.run

🔧 ETAPA 5: Implementação de get_options(var) para os Selectboxes
Nesta célula, vamos definir a função get_options(var) que retorna a lista de opções para cada campo do formulário, usando o VARIABLE_MAP e, quando disponível, extraindo do dataset treinado (por exemplo um CSV de categorias).
Objetivo: fornecer as opções corretas a cada st.selectbox sem hard-code.

In [19]:
import pandas as pd
from mapping import VARIABLE_MAP

# 1️⃣ Carrega o dataset de referência (categorias originais)
df = pd.read_csv("/workspace/data/curated/train_curated_v2.csv")

# 2️⃣ Constroi um dicionário de opções a partir dos dados
OPTIONS_MAP = {}
for var, cfg in VARIABLE_MAP.items():
    if cfg["widget"] == "selectbox":
        # se for coluna numérica contínua, opcional: discretize ou forneça faixas
        if pd.api.types.is_numeric_dtype(df[var]):
            OPTIONS_MAP[var] = sorted(df[var].unique().tolist())
        else:
            OPTIONS_MAP[var] = sorted(df[var].dropna().unique().tolist())

# 3️⃣ Define a função get_options
def get_options(var_name: str):
    """Retorna lista de opções para o selectbox de var_name."""
    return OPTIONS_MAP.get(var_name, [])

# Verificação
print("✅ Opções carregadas para exemplos:", 
      {k: OPTIONS_MAP[k][:5] for k in list(OPTIONS_MAP)[:5]})


✅ Opções carregadas para exemplos: {}


O resultado {} indica que nenhum campo em VARIABLE_MAP tinha cfg["widget"] == "selectbox", logo o OPTIONS_MAP ficou vazio.
Próximo passo: atualizar automaticamente o VARIABLE_MAP (e portanto o arquivo mapping.py) com os label, widget e api extraídos do seu streamlit_app_V1.py, para que o loop de geração de opções encontre de fato os campos mapeados.




In [20]:
import os, re, json
from mapping import VARIABLE_MAP

# 1️⃣ Associa cada label do formulário à variável do pipeline
UI_LABEL_TO_VAR = {
    'Idade': 'Age',
    'Ocupação': 'Occupation',
    'Perfil de Crédito': 'Credit_Mix',
    'Pagamento Mínimo': 'Payment_of_Min_Amount',
    'Idade do Histórico de Crédito': 'Credit_History_Age_Months',
    'Salário Mensal': 'Monthly_Inhand_Salary',
    'Nº de Cartões': 'Num_Credit_Card',
    'Pagamentos Atrasados': 'Num_of_Delayed_Payment',
    'Renda Anual': 'Annual_Income',
    'Dívida': 'Outstanding_Debt',
    'Taxa de Juros': 'Interest_Rate',
    'Mudança de Limite': 'Changed_Credit_Limit',
    'Histórico Binned': 'Credit_History_Age_Binned',
    'Utilização de Crédito': 'Credit_Utilization_Ratio',
    'Dias de Atraso': 'Delay_from_due_date',
    'Balanço Mensal': 'Monthly_Balance',
    'Nº de Contas Bancárias': 'Num_Bank_Accounts',
    'Consultas de Crédito': 'Num_Credit_Inquiries',
    'Nº de Empréstimos': 'Num_of_Loan',
    'EMI Mensal': 'Total_EMI_per_month',
    'Tipo de Empréstimo': 'Type_of_Loan',
    'Investimento Mensal': 'Amount_invested_monthly'
}

# 2️⃣ Lê o código do seu Streamlit e detecta widget para cada label
with open('/workspace/streamlit_app_V1.py', 'r', encoding='utf-8') as f:
    code = f.read()

for label, var in UI_LABEL_TO_VAR.items():
    # procura st.selectbox ou st.number_input com este label
    m = re.search(rf"st\.(selectbox|number_input)\(\s*label=['\"]{re.escape(label)}['\"]", code)
    if m and var in VARIABLE_MAP:
        VARIABLE_MAP[var]['label']  = label
        VARIABLE_MAP[var]['widget'] = m.group(1)
        VARIABLE_MAP[var]['api']    = var

# 3️⃣ Regrava mapping.py com o novo VARIABLE_MAP preenchido
mapping_path = '/workspace/mapping.py'
with open(mapping_path, 'w', encoding='utf-8') as f:
    f.write('# Auto-generated mapping module\nVARIABLE_MAP = ')
    json.dump(VARIABLE_MAP, f, indent=4, ensure_ascii=False)

print(f"✅ mapping.py atualizado em: {mapping_path}")


✅ mapping.py atualizado em: /workspace/mapping.py


🔧 ETAPA 5: Reconstrução de OPTIONS_MAP e get_options
Com o mapping.py agora devidamente preenchido, reconstruiremos o dicionário OPTIONS_MAP a partir do dataset de referência, garantindo que para cada variável com widget == "selectbox" existam opções reais. Em seguida, definiremos a função get_options para uso nos st.selectbox.

In [21]:
import pandas as pd
from mapping import VARIABLE_MAP

# COMANDO PRINCIPAL: geração de OPTIONS_MAP a partir do CSV treinado
csv_path = "/workspace/data/curated/train_curated_v2.csv"
df = pd.read_csv(csv_path)

OPTIONS_MAP = {
    var: sorted(df[var].dropna().unique().tolist())
    for var, cfg in VARIABLE_MAP.items()
    if cfg.get("widget") == "selectbox"
}

# Função auxiliar para o Streamlit
def get_options(var_name: str):
    return OPTIONS_MAP.get(var_name, [])

# Verificação de exemplo
print("✅ OPTIONS_MAP preenchido com exemplos:",
      {k: OPTIONS_MAP[k][:5] for k in list(OPTIONS_MAP)[:5]})


✅ OPTIONS_MAP preenchido com exemplos: {}


🔧 ETAPA 6: Recarregar mapping.py e reconstruir OPTIONS_MAP
Nesta célula, vamos recarregar o módulo mapping para capturar as alterações recentes em VARIABLE_MAP (widgets e labels preenchidos). Em seguida, reconstruiremos o dicionário OPTIONS_MAP a partir do CSV de treino, garantindo que apenas as variáveis com widget == "selectbox" apareçam.

In [23]:
# COMANDO PRINCIPAL: corrige null → None em mapping.py
from pathlib import Path

mapping_path = Path("/workspace/mapping.py")
code = mapping_path.read_text(encoding="utf-8")

# substitui o literal JSON null pelo literal Python None
corrected = code.replace(": null", ": None")

# grava de volta
mapping_path.write_text(corrected, encoding="utf-8")

# verificação rápida: tenta importar
import importlib, mapping
importlib.reload(mapping)
print("✅ mapping.py corrigido e módulo carregado com sucesso.")


✅ mapping.py corrigido e módulo carregado com sucesso.


In [24]:
import importlib
import pandas as pd
import mapping

# 1️⃣ Recarrega o módulo mapping para obter o VARIABLE_MAP atualizado
importlib.reload(mapping)
VARIABLE_MAP = mapping.VARIABLE_MAP

# 2️⃣ Carrega o dataset de referência
df = pd.read_csv("/workspace/data/curated/train_curated_v2.csv")

# 3️⃣ Reconstrói OPTIONS_MAP apenas para selectboxes
OPTIONS_MAP = {
    var: sorted(df[var].dropna().unique().tolist())
    for var, cfg in VARIABLE_MAP.items()
    if cfg.get("widget") == "selectbox"
}

# 4️⃣ Define get_options
def get_options(var_name: str):
    return OPTIONS_MAP.get(var_name, [])

# 5️⃣ Verificação
print("✅ OPTIONS_MAP agora possui campos:", list(OPTIONS_MAP.keys()))


✅ OPTIONS_MAP agora possui campos: []


In [25]:
from mapping import VARIABLE_MAP

# COMANDO PRINCIPAL: listar variáveis que têm widget configurado
configured = {var: cfg for var, cfg in VARIABLE_MAP.items() if cfg.get("widget") is not None}
print("📋 Campos com widget definido:", configured)


📋 Campos com widget definido: {}


🔧 ETAPA 6.2: População Automática de label, widget e api em mapping.py
Nesta etapa, vamos varrer o código do Streamlit (streamlit_app_V1.py), identificar cada widget e seu rótulo, e preencher o VARIABLE_MAP em mapping.py de forma programática. Depois recarregamos o módulo para que as entradas passem a ter widget != None.




In [26]:
import re
from pathlib import Path
import importlib

# 1️⃣ Define o mapeamento label → variável do pipeline
UI_LABEL_TO_VAR = {
    'Idade': 'Age',
    'Ocupação': 'Occupation',
    'Perfil de Crédito': 'Credit_Mix',
    'Pagamento Mínimo': 'Payment_of_Min_Amount',
    'Idade do Histórico de Crédito': 'Credit_History_Age_Months',
    'Salário Mensal': 'Monthly_Inhand_Salary',
    'Nº de Cartões': 'Num_Credit_Card',
    'Pagamentos Atrasados': 'Num_of_Delayed_Payment',
    'Renda Anual': 'Annual_Income',
    'Dívida': 'Outstanding_Debt',
    'Taxa de Juros': 'Interest_Rate',
    'Mudança de Limite': 'Changed_Credit_Limit',
    'Histórico Binned': 'Credit_History_Age_Binned',
    'Utilização de Crédito': 'Credit_Utilization_Ratio',
    'Dias de Atraso': 'Delay_from_due_date',
    'Balanço Mensal': 'Monthly_Balance',
    'Nº de Contas Bancárias': 'Num_Bank_Accounts',
    'Consultas de Crédito': 'Num_Credit_Inquiries',
    'Nº de Empréstimos': 'Num_of_Loan',
    'EMI Mensal': 'Total_EMI_per_month',
    'Tipo de Empréstimo': 'Type_of_Loan',
    'Investimento Mensal': 'Amount_invested_monthly'
}

# 2️⃣ Carrega o código Streamlit
streamlit_path = Path("/workspace/streamlit_app_V1.py")
code = streamlit_path.read_text(encoding="utf-8")

# 3️⃣ Atualiza mapping.py
mapping_path = Path("/workspace/mapping.py")
from mapping import VARIABLE_MAP

for label, var in UI_LABEL_TO_VAR.items():
    # busca tanto selectbox quanto number_input
    m = re.search(rf"st\.(selectbox|number_input)\(\s*label=['\"]{re.escape(label)}['\"]", code)
    if m and var in VARIABLE_MAP:
        VARIABLE_MAP[var]["label"]  = label
        VARIABLE_MAP[var]["widget"] = m.group(1)
        VARIABLE_MAP[var]["api"]    = var

# 4️⃣ Salva o mapping corrigido
mapping_code = "# Auto-generated mapping module\nVARIABLE_MAP = " + repr(VARIABLE_MAP)
mapping_path.write_text(mapping_code, encoding="utf-8")

# 5️⃣ Recarrega e verifica
import mapping
importlib.reload(mapping)
print("✅ Campos com widget agora definidos:", 
      [v for v,c in mapping.VARIABLE_MAP.items() if c.get("widget") is not None])


✅ Campos com widget agora definidos: []


In [27]:
from mapping import VARIABLE_MAP

# COMANDO PRINCIPAL: listar as primeiras chaves de VARIABLE_MAP
chaves = list(VARIABLE_MAP.keys())
print("🔑 Chaves em VARIABLE_MAP (exemplo):", chaves[:10])
print("🔢 Total de variáveis mapeadas:", len(chaves))


🔑 Chaves em VARIABLE_MAP (exemplo): ['Age', 'Age_Binned', 'Amount_invested_monthly', 'Amount_invested_monthly_Binned', 'Amount_invested_monthly_Binned_High', 'Amount_invested_monthly_Binned_Low', 'Amount_invested_monthly_Binned_Moderate', 'Annual_Income', 'Annual_Income_Binned', 'Changed_Credit_Limit']
🔢 Total de variáveis mapeadas: 92


In [28]:
import re
from pathlib import Path
from mapping import VARIABLE_MAP

# COMANDO PRINCIPAL: extrair todos os labels usados em st.selectbox/number_input
code = Path("/workspace/streamlit_app_V1.py").read_text(encoding="utf-8")
ui_labels = re.findall(r"st\.(?:selectbox|number_input)\(\s*label=['\"](.+?)['\"]", code)

# 1️⃣ Labels únicos extraídos do UI
ui_labels = list(dict.fromkeys(ui_labels))
print("🎨 Labels do Streamlit:", ui_labels)

# 2️⃣ Variáveis do pipeline disponíveis
pipeline_vars = set(VARIABLE_MAP.keys())
print("🔑 Variáveis no VARIABLE_MAP:", list(pipeline_vars)[:10], "... total:", len(pipeline_vars))

# 3️⃣ Identifica labels sem mapeamento e variáveis sem widget
labels_sem_map = [lbl for lbl in ui_labels if lbl not in globals().get("UI_LABEL_TO_VAR", {})]
vars_sem_widget = [var for var, cfg in VARIABLE_MAP.items() if cfg.get("widget") is None]

print("❗ Labels sem entrada em UI_LABEL_TO_VAR:", labels_sem_map)
print("❗ Variáveis sem widget definido em mapping.py (ainda sem preencher):", vars_sem_widget[:10], "... total:", len(vars_sem_widget))


🎨 Labels do Streamlit: []
🔑 Variáveis no VARIABLE_MAP: ['Num_of_Loan_Binned', 'Monthly_Balance_Binned_Moderate', 'Type_of_Loan_Category_Debt Consolidation Loan', 'Changed_Credit_Limit_Binned', 'Monthly_Inhand_Salary', 'Monthly_Balance', 'Type_of_Loan_Category_Home Equity Loan', 'Num_Credit_Inquiries', 'Occupation_Group_Writer', 'Credit_Utilization_Ratio'] ... total: 92
❗ Labels sem entrada em UI_LABEL_TO_VAR: []
❗ Variáveis sem widget definido em mapping.py (ainda sem preencher): ['Age', 'Age_Binned', 'Amount_invested_monthly', 'Amount_invested_monthly_Binned', 'Amount_invested_monthly_Binned_High', 'Amount_invested_monthly_Binned_Low', 'Amount_invested_monthly_Binned_Moderate', 'Annual_Income', 'Annual_Income_Binned', 'Changed_Credit_Limit'] ... total: 92


In [29]:
from pathlib import Path

# COMANDO PRINCIPAL: imprimir as primeiras 200 linhas de streamlit_app_V1.py
lines = Path("/workspace/streamlit_app_V1.py").read_text(encoding="utf-8").splitlines()
print("📄 Primeiras 200 linhas do Streamlit App:")
for i, line in enumerate(lines[:200], start=1):
    print(f"{i:03d}: {line}")


📄 Primeiras 200 linhas do Streamlit App:
001: # 🔧 ETAPA: Integração do dicionário de mapeamento e função de conversão no Streamlit
002: 
003: import streamlit as st
004: import requests
005: 
006: # 1️⃣ Dicionário de mapeamentos para valores categóricos
007: mapeamentos = {
008:     "Age_Binned": {
009:         "20-30": 0, "30-40": 1, "40-50": 2, "50-60": 3, "60-70": 4, "70+": 5
010:     },
011:     "Amount_invested_monthly_Binned": {
012:         "0-500": 0, "500-1000": 1, "1000-1500": 2, "1500-2000": 3, "2000+": 4
013:     },
014:     "Annual_Income_Binned": {
015:         "0-50K": 0, "50K-100K": 1, "100K-150K": 2, "150K-200K": 3, "200K+": 4
016:     },
017:     "Changed_Credit_Limit_Binned": {
018:         "0-2K": 0, "2K-5K": 1, "5K-10K": 2, "10K+": 3
019:     },
020:     "Credit_History_Age_Binned": {
021:         "0-5Y": 0, "5Y-10Y": 1, "10Y-15Y": 2, "15Y+": 3
022:     },
023:     "Credit_Mix": {
024:         "Bad": 0, "Standard": 1, "Good": 2
025:     },
026:     "Credit_Utilizat

In [30]:
from pathlib import Path
import re
import mapping

# 1️⃣ Lê o Streamlit e extrai (var, widget, label)
streamlit = Path("/workspace/streamlit_app_V1.py").read_text(encoding="utf-8")
pattern = re.compile(r'dados\["(?P<var>[^"]+)"\]\s*=\s*st\.(?P<widget>\w+)\("(?P<label>[^"]+)"')
entries = pattern.finditer(streamlit)

# 2️⃣ Atualiza mapping.VARIABLE_MAP
for m in entries:
    var = m.group("var")
    mapping.VARIABLE_MAP[var]["label"]  = m.group("label")
    mapping.VARIABLE_MAP[var]["widget"] = m.group("widget")
    mapping.VARIABLE_MAP[var]["api"]    = var

# 3️⃣ Sobrescreve mapping.py com o dicionário preenchido
out = "VARIABLE_MAP = " + repr(mapping.VARIABLE_MAP)
Path("/workspace/mapping.py").write_text(out, encoding="utf-8")

print("✅ mapping.py atualizado com labels, widgets e api para todas as variáveis.")


✅ mapping.py atualizado com labels, widgets e api para todas as variáveis.


In [33]:
import mlflow

# ▪️ Defina aqui seu run_id (o mesmo que você usou para carregar o modelo)
run_id = "7077ebfbf696487384bd5a59034170c5"  

# ▪️ Carrega o modelo congelado e extrai a assinatura
modelo_congelado = mlflow.pyfunc.load_model(f"runs:/{run_id}/model")
signature = modelo_congelado.metadata.signature


In [34]:
# 🔧 ETAPA 4: Geração Dinâmica do Modelo Pydantic a partir de VARIABLE_MAP

from pydantic import create_model
from mlflow.types import DataType
from mapping import VARIABLE_MAP

# 1️⃣ Prepara os campos para create_model
fields = {}
for var_name, cfg in VARIABLE_MAP.items():
    api_name = cfg["api"]
    if not api_name:
        continue
    # encontra o tipo MLflow para esta variável
    mlflow_input = next(f for f in signature.inputs if f.name == var_name)
    dtype = mlflow_input.type
    # mapeia para tipo Python
    if dtype in (DataType.double, DataType.long):
        py_type = float
    elif dtype == DataType.boolean:
        py_type = bool
    else:
        py_type = str
    fields[api_name] = (py_type, ...)

# 2️⃣ Cria o modelo Pydantic
RequestPayload = create_model("RequestPayload", **fields)

# 3️⃣ Verificação
print("✅ RequestPayload criado com campos:", list(RequestPayload.__fields__.keys()))


✅ RequestPayload criado com campos: ['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Payment_Binned', 'Num_of_Loan_Binned', 'Occupation', 'Outstanding_Debt_Binned', 'Payment_of_Min_Amount', 'Total_EMI_per_month_Binned', 'Type_of_Loan']


/tmp/ipykernel_11246/2647487685.py:29: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print("✅ RequestPayload criado com campos:", list(RequestPayload.__fields__.keys()))


In [37]:
import mlflow

# 1️⃣ Carrega o modelo via model_uri
model = mlflow.pyfunc.load_model(model_uri)

# 2️⃣ Extrai a assinatura
signature = model.metadata.signature

# 3️⃣ Lista cada variável de entrada e seu tipo
print("🗂️ signature.inputs:")
for inp in signature.inputs:
    print(f"• {inp.name:<30} | Tipo MLflow: {inp.type}")


🗂️ signature.inputs:
• Age                            | Tipo MLflow: DataType.double
• Age_Binned                     | Tipo MLflow: DataType.string
• Amount_invested_monthly        | Tipo MLflow: DataType.double
• Amount_invested_monthly_Binned | Tipo MLflow: DataType.string
• Amount_invested_monthly_Binned_High | Tipo MLflow: DataType.boolean
• Amount_invested_monthly_Binned_Low | Tipo MLflow: DataType.boolean
• Amount_invested_monthly_Binned_Moderate | Tipo MLflow: DataType.boolean
• Annual_Income                  | Tipo MLflow: DataType.double
• Annual_Income_Binned           | Tipo MLflow: DataType.string
• Changed_Credit_Limit           | Tipo MLflow: DataType.double
• Changed_Credit_Limit_Binned    | Tipo MLflow: DataType.string
• Credit_History_Age             | Tipo MLflow: DataType.string
• Credit_History_Age_Binned      | Tipo MLflow: DataType.string
• Credit_History_Age_Months      | Tipo MLflow: DataType.double
• Credit_Mix                     | Tipo MLflow: DataType.strin

In [38]:
# 1️⃣ Extrai nomes da signature e do mapping
signature_vars = [inp.name for inp in signature.inputs]
mapping_vars   = list(VARIABLE_MAP.keys())

# 2️⃣ Identifica inconsistências
missing_in_map = set(signature_vars) - set(mapping_vars)
extra_in_map   = set(mapping_vars)   - set(signature_vars)

# 3️⃣ Exibe resultado
print("❗ Variáveis da signature QUE NÃO ESTÃO no VARIABLE_MAP:", missing_in_map)
print("❗ Variáveis no VARIABLE_MAP QUE NÃO aparecem na signature:", extra_in_map)


❗ Variáveis da signature QUE NÃO ESTÃO no VARIABLE_MAP: set()
❗ Variáveis no VARIABLE_MAP QUE NÃO aparecem na signature: set()


In [39]:
# 🔧 ETAPA 5: geração dinâmica do RequestPayload dentro da API
from fastapi       import FastAPI, HTTPException
from pydantic      import create_model
from mlflow.pyfunc import load_model
from mlflow.types  import DataType
from mapping       import VARIABLE_MAP
from typing        import List

# — 1️⃣ Carrega o modelo (use o model_uri ou run_id que você já declarou)
model = load_model(f"runs:/{run_id}/model")

# — 2️⃣ (Re)cria dinamicamente o Pydantic RequestPayload
fields = {}
for var, cfg in VARIABLE_MAP.items():
    api_name = cfg["api"]
    if not api_name:
        continue
    # encontra o tipo MLflow correspondente
    ml_type = next(f.type for f in signature.inputs if f.name == var)
    py_type = (
        float if ml_type in (DataType.double, DataType.long)
        else bool  if ml_type is DataType.boolean
        else str
    )
    fields[api_name] = (py_type, ...)
RequestPayload = create_model("RequestPayload", **fields)

# — 3️⃣ Monta o FastAPI usando o modelo dinamicamente gerado
app = FastAPI()

@app.post("/predict")
async def predict(payload: RequestPayload):
    # converte para dicionário simples
    data_dict = payload.model_dump()
    try:
        # o MLflow espera uma lista de registros
        result = model.predict([data_dict])
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    return {"prediction": int(result[0])}


In [40]:
# 🔧 ETAPA 6: Streamlit dinâmico via VARIABLE_MAP
import streamlit as st
import requests
from mapping import VARIABLE_MAP

st.title("Classificador de Crédito")

with st.form("input_form"):
    dados = {}
    # Para cada variável mapeada, gera dinamicamente o widget
    for var_name, cfg in VARIABLE_MAP.items():
        label  = cfg["label"]
        widget = cfg["widget"]
        api    = cfg["api"]
        # só cria se tiver widget configurado
        if not (label and widget and api):
            continue

        if widget == "selectbox":
            options = cfg.get("options", [])
            dados[api] = st.selectbox(label, options, key=api)
        elif widget == "number_input":
            min_, max_ = cfg.get("min", 0), cfg.get("max", None)
            dados[api] = st.number_input(label, min_value=min_, max_value=max_, key=api)
        elif widget == "checkbox":
            dados[api] = st.checkbox(label, key=api)
        # (adicione aqui outros widgets conforme necessário)

    submitted = st.form_submit_button("Enviar")

if submitted:
    # converte valores categóricos antes de enviar
    from mapping import OPTIONS_MAP
    def converter(d):
        out = {}
        for k, v in d.items():
            if k in OPTIONS_MAP:
                out[k] = OPTIONS_MAP[k][v]
            else:
                out[k] = v
        return out

    payload = converter(dados)
    st.json(payload)

    resp = requests.post("http://localhost:8000/predict",
                         json=payload,
                         headers={"x-api-key": "quantum123"})
    if resp.ok:
        st.success(f"✅ Predição: {resp.json()['prediction']}")
    else:
        st.error(f"❌ {resp.status_code}: {resp.text}")


2025-07-23 20:16:08.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:16:08.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:16:08.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:16:08.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:16:08.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:16:08.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:16:08.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 20:16:08.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar